# Azimuth oscillations 

Repeating the azimuth oscillations in preparation for trying the \
"go past and come back" idea.
Craig Lage - 22Sep23

In [ ]:
import asyncio
import time
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.time import Time

from lsst.ts import salobj

from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS
from lsst.ts.observatory.control.utils import RotType

from lsst_efd_client import EfdClient

In [ ]:
client = EfdClient('summit_efd')

In [ ]:
logger = logging.getLogger(f"Daytime Checkout {Time.now()} UT")
logger.level = logging.DEBUG

In [ ]:
logger.info(os.environ["OSPL_URI"])
logger.info(os.environ["LSST_DDS_PARTITION_PREFIX"])

### Instantiate the control classes

In [ ]:
domain = salobj.Domain()
atcs = ATCS(domain)
latiss = LATISS(domain)
await asyncio.gather(atcs.start_task, latiss.start_task)
await atcs.assert_liveliness()

If ATCamera and/or ATOODS don't come up, see section `Quick Troubleshooting` for support commands. 

### Bias verification

In [ ]:
for i in range(5):
    await latiss.take_bias(nbias=1)
    await asyncio.sleep(2.0)

In [ ]:
await atcs.disable_dome_following()

### Now run the actual test, taking a 30 second image at each location.

In [ ]:
# Move to starting location
coord=atcs.radec_from_azel(az=270, el=45)
await atcs.slew_icrs(coord.ra, coord.dec, rot=0.0, rot_type=RotType.Physical)

In [ ]:
# Now run the actual test 
# First image is 2023092200006

az = 270.0
await latiss.take_darks(30.0, 1)
for i in range(90):
    az -= 2.0
    coord=atcs.radec_from_azel(az=az, el=45)
    await atcs.slew_icrs(coord.ra, coord.dec, rot=0.0, rot_type=RotType.Physical)
    await latiss.take_darks(30.0, 1)
for i in range(90):
    az += 2.0
    coord=atcs.radec_from_azel(az=az, el=45)
    await atcs.slew_icrs(coord.ra, coord.dec, rot=0.0, rot_type=RotType.Physical)
    await latiss.take_darks(30.0, 1)


In [ ]:
# Now run the actual test with go-and-come-back only while slewing negative in Az.
# First image is 2023092200187
deltaAz = 2.0
az = 270.0
await latiss.take_darks(30.0, 1)
for i in range(90):
    az -= 2.0
    coord=atcs.radec_from_azel(az=az-deltaAz, el=45)
    await atcs.slew_icrs(coord.ra, coord.dec, rot=0.0, rot_type=RotType.Physical)
    await asyncio.sleep(2.0)
    coord=atcs.radec_from_azel(az=az, el=45)
    await atcs.slew_icrs(coord.ra, coord.dec, rot=0.0, rot_type=RotType.Physical)
    await latiss.take_darks(30.0, 1)
for i in range(90):
    az += 2.0
    coord=atcs.radec_from_azel(az=az, el=45)
    await atcs.slew_icrs(coord.ra, coord.dec, rot=0.0, rot_type=RotType.Physical)
    await latiss.take_darks(30.0, 1)


### Stopping early because out of time.

In [ ]:
# Stop tracking
await atcs.stop_tracking()

In [ ]:
# Get Rubin TV JSON
filename = f'/scratch/cslage/starTracker/auxtel_2023-09-22.json'
df = pd.read_json(filename)
df = df.transpose()

In [ ]:
%matplotlib inline
fig, axs = plt.subplots(2,1,figsize = (12,8))
plt.subplots_adjust(hspace=0.5)
images = [[6, 186, "2023-09-22", "Baseline"], [187, 293, "2023-09-22", "Go-and-come-back"]]
for i, [firstImage, lastImage, dayObs, name] in enumerate(images):
    start = Time(f"{dayObs}T{df.loc[firstImage]['TAI']}", format='isot', scale='tai')
    end = Time(f"{dayObs}T{df.loc[lastImage]['TAI']}", format='isot', scale='tai')
    az = await client.select_packed_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", "azimuthCalculatedAngle",\
                                                start.utc, end.utc)
    azs = az['azimuthCalculatedAngle'].values
    times = az['times'].values
    t0 = times[0]
    times -= t0
    tmax = times[-1]

    axs[i].set_title(f"{name} - {dayObs}", fontsize=24)
    axs[i].plot(times, azs)
    axs[i].set_ylabel("Azimuth(degrees)")
    axs[i].set_xlabel("Time(seconds)")
    axs[i].set_xlim(0,9000)
    ax2 = axs[i].twinx()
    mount_times = []
    mount_motions = []
    for seqNum in df.index.values.tolist():
        if seqNum >= firstImage and seqNum <= lastImage:
            time = Time(f"{dayObs}T{df.loc[seqNum]['TAI']}", format='isot', scale='tai')
            mount_times.append(time.utc.unix_tai - t0)
            mount_motions.append(df.loc[seqNum]['Mount motion image degradation'])
    ax2.scatter(mount_times, mount_motions, marker='x', color='g')
    ax2.set_ylabel("Mount jitter (arcseconds)")
    ax2.set_ylim(0, 1.5)
plt.savefig(f"/home/craiglagegit/DATA/Go-and-Come-Back_Test_22Sep23.png")
